# 结构化数据分类

本教程展示了如何对结构化数据进行分类（例如CSV中的表格数据）。我们使用Keras定义模型，并将csv中各列的特征转化为训练的输入。 本教程包含一下功能代码：

- 使用Pandas加载CSV文件
- 构建一个输入的pipeline，使用tf.data批处理和打乱数据
- 从CSV中的列映射到用于训练模型的输入要素
- 使用Keras构建，训练和评估模型

In [1]:
from __future__ import absolute_import, division, print_function

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.0.0-rc0


## 数据集

我们将使用克利夫兰诊所心脏病基金会提供的一个小数据集。 CSV中有几百行。 每行描述一个患者，每列描述一个属性。 我们将使用此信息来预测患者是否患有心脏病，该疾病在该数据集中是二元分类任务

|Column|Description|Feature Type|Data Type|
|:-:|:-:|:-:|:-:|
|Age|Age in years|Numerical|integer|
|Sex|(1 = male; 0 = female)|Categorical|integer|
|CP|Chest pain type (0, 1, 2, 3, 4)|Categorical|integer|
|Trestbpd|Resting blood pressure (in mm Hg on admission to the hospital)|Numerical|integer|
|Chol|Serum cholestoral in mg/dl|Numerical|integer|
|FBS|(fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)|Categorical|integer
|RestECG|Resting electrocardiographic results (0, 1, 2)|Categorical|integer
|Thalach|Maximum heart rate achieved|Numerical|integer|
|Exang|Exercise induced angina (1 = yes; 0 = no)|Categorical|integer|
|Oldpeak|ST depression induced by exercise relative to rest|Numerical|integer|
|Slope|The slope of the peak exercise ST segment|Numerical|float|
|CA|Number of major vessels (0-3) colored by flourosopy|Numerical|integer|
|Thal|3 = normal; 6 = fixed defect; 7 = reversable defect|Categorical|string|
|Target|Diagnosis of heart disease (1 = true; 0 = false)|Classification|integer

## 准备数据

使用pandas读取数据

In [3]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


划分训练集验证集和测试集

In [4]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


使用 tf.data 构造输入 pipeline

In [6]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [7]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of ages:', feature_batch['age'])
    print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([35 59 35 68 58], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([0 1 0 1 1], shape=(5,), dtype=int32)


## tensorflow 的 feature column

In [8]:
example_batch = next(iter(train_ds))[0]

In [9]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

### 数字列

特征列的输出成为模型的输入。 数字列是最简单的列类型。 它用于表示真正有价值的特征。 使用此列时，模型将从数据框中接收未更改的列值。

In [10]:
age = feature_column.numeric_column("age")
demo(age)

W0903 16:41:37.129968 140735620006784 base_layer.py:1772] Layer dense_features is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



[[59.]
 [43.]
 [53.]
 [54.]
 [59.]]


### Bucketized 列（桶列）

通常，您不希望将数字直接输入模型，而是根据数值范围将其值分成不同的类别。 考虑代表一个人年龄的原始数据。 我们可以使用bucketized列将年龄分成几个桶，而不是将年龄表示为数字列。 请注意，下面的one-hot描述了每行匹配的年龄范围。

In [11]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 50])
demo(age_buckets)

W0903 16:44:09.256133 140735620006784 base_layer.py:1772] Layer dense_features_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



[[0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]]


### 类别列

在该数据集中，thal 表示为字符串（例如“固定”，“正常”或“可逆”）。 我们无法直接将字符串提供给模型。 相反，我们必须首先将它们映射到数值。 类别列提供了一种将字符串表示为单热矢量的方法（就像上面用年龄段看到的那样）。 类别表可以使用 categorical_column_with_vocabulary_list 作为列表传递，或者使用 categorical_column_with_vocabulary_file 从文件加载。

In [12]:
thal = feature_column.categorical_column_with_vocabulary_list('thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

W0903 16:49:49.580356 140735620006784 base_layer.py:1772] Layer dense_features_2 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W0903 16:49:49.592890 140735620006784 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:4273: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.fea

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


### 嵌入列

假设我们不是只有几个可能的字符串，而是每个类别有数千（或更多）值。 由于多种原因，随着类别数量的增加，使用单热编码训练神经网络变得不可行。 我们可以使用嵌入列来克服此限制。 嵌入列不是将数据表示为多维度的单热矢量，而是将数据表示为低维密集向量，其中每个单元格可以包含任意数字，而不仅仅是0或1.嵌入的大小是必须训练调整的参数。

注：当分类列具有许多可能的值时，最好使用嵌入列。

In [13]:
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

W0903 16:51:39.677568 140735620006784 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:353: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
W0903 16:51:39.686568 140735620006784 base_layer.py:1772] Layer dense_features_3 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If

[[ 0.09201286  0.24908379 -0.32961792  0.22948322  0.3095203  -0.05739767
  -0.37226698 -0.08391729]
 [ 0.09201286  0.24908379 -0.32961792  0.22948322  0.3095203  -0.05739767
  -0.37226698 -0.08391729]
 [-0.05352275 -0.02077207 -0.3797898   0.08031797  0.11485706  0.12569235
   0.13541797 -0.03527887]
 [-0.05352275 -0.02077207 -0.3797898   0.08031797  0.11485706  0.12569235
   0.13541797 -0.03527887]
 [-0.05352275 -0.02077207 -0.3797898   0.08031797  0.11485706  0.12569235
   0.13541797 -0.03527887]]


### 哈希特征列

表示具有大量值的分类列的另一种方法是使用 categorical_column_with_hash_bucket。 此功能列计算输入的哈希值，然后选择一个 hash_bucket_size 存储桶来编码字符串。 使用此列时，您不需要提供词汇表，并且可以选择使 hash_buckets 的数量远远小于实际类别的数量以节省空间。

注：该技术的一个重要缺点是可能存在冲突，其中不同的字符串被映射到同一个桶。

In [15]:
thal_hashed = feature_column.categorical_column_with_hash_bucket('thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

W0903 16:53:15.199796 140735620006784 base_layer.py:1772] Layer dense_features_4 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W0903 16:53:15.207937 140735620006784 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:4328: HashedCategoricalColumn._num_buckets (from tensorflow.python.feature_colum

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### 交叉功能列

将特征组合成单个特征（更好地称为特征交叉），使模型能够为每个特征组合学习单独的权重。 在这里，我们将创建一个与 age 和 thal 交叉的新功能。 请注意，crossed_column 不会构建所有可能组合的完整表（可能非常大）。 相反，它由 hashed_column 支持，因此您可以选择表的大小。

In [16]:
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))

W0903 16:54:32.991508 140735620006784 base_layer.py:1772] Layer dense_features_5 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W0903 16:54:33.003357 140735620006784 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:4328: CrossedColumn._num_buckets (from tensorflow.python.feature_column.feature_

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 选择使用 feature column

In [17]:
feature_columns = []

# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
    feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

构建特征层

In [18]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [19]:
batch_size = 32

train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

## 构建模型并训练

In [20]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=5)

W0903 16:57:50.974342 140735620006784 base_layer.py:1772] Layer sequential is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch 1/5
7/7 [==============================] - 1s 214ms/step - loss: 2.2090 - accuracy: 0.5596 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
7/7 [==============================] - 0s 7ms/step - loss: 0.8664 - accuracy: 0.6684 - val_loss: 0.4780 - val_accuracy: 0.7551
Epoch 3/5
7/7 [==============================] - 0s 8ms/step - loss: 0.6681 - accuracy: 0.7254 - val_loss: 0.7240 - val_accuracy: 0.7143
Epoch 4/5
7/7 [==============================] - 0s 8ms/step - loss: 0.6654 - accuracy: 0.6891 - val_loss: 0.4497 - val_accuracy: 0.7551
Epoch 5/5
7/7 [==============================] - 0s 9ms/step - loss: 0.5890 - accuracy: 0.7513 - val_loss: 0.4524 - val_accuracy: 0.7551


## 测试

In [21]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy, "Loss", loss)

2/2 [==============================] - 0s 4ms/step - loss: 0.6015 - accuracy: 0.6393
Accuracy 0.6393443 Loss 0.6014837324619293
